# Модуль 4 (RAG) — пошаговый учебник «как для детей»

**Зачем этот блокнот:** объяснить весь модуль 4 очень простыми словами, шаг за шагом, с короткими примерами.

**Как читать:** выполняйте сверху вниз, даже если уже знакомы с темой. Здесь всё разложено «по полочкам».

## 0. Что такое RAG (очень коротко)

Представь, что у тебя есть **умный помощник** (LLM), но он **не знает все ваши документы**.

**RAG** — это способ **дать модели нужные документы прямо перед ответом**.

Простая схема:
1. Вопрос пользователя.
2. Поиск по базе документов.
3. Найденные кусочки текста кладём в промпт.
4. LLM отвечает **уже с контекстом**.

Именно так мы и работали в модуле 4.

## 1. Подготовка среды (Colab)

**Правило №1:** все библиотеки ставим в блокноте.

Почему? В Colab версии часто конфликтуют. Мы фиксируем нужные версии, чтобы всё работало стабильно.

In [ ]:
# Установка библиотек (минимально необходимое)
%pip -q install -U \
  langchain \
  langchain-community \
  langchain-openai \
  chromadb \
  rank-bm25 \
  matplotlib \
  sentence-transformers \
  pydantic==2.12.3 \
  requests==2.32.4

### Если Colab ругается на зависимости

Иногда в Colab уже стоят «несовместимые» пакеты. Тогда мы фиксируем версии `opentelemetry`.
Это не про логику RAG — это **просто ремонт окружения**.

In [ ]:
# Фиксация opentelemetry при конфликтах
%pip -q install -U \
  opentelemetry-api==1.37.0 \
  opentelemetry-sdk==1.37.0 \
  opentelemetry-proto==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0 \
  opentelemetry-exporter-otlp-proto-grpc==1.37.0

## 2. Предобработка данных (очистка)

**Очень просто:**
- удаляем мусор (пустые строки, «служебные» куски),
- выравниваем текст (лишние пробелы),
- убираем дубликаты.

**Зачем?**
Если мусор попадёт в базу, поиск станет хуже.

В модуле мы сделали отдельный блокнот про это: `00_data_preprocessing.ipynb`.

## 3. Загрузка документов и разбиение на кусочки

Текст «целиком» слишком большой. Мы режем его на маленькие куски.

**Почему?**
LLM не сможет проглотить всю книгу. Но сможет прочитать 3–5 нужных кусочков.

В модуле это блокнот: `01_load_split.ipynb`.

Ключевые параметры:
- `chunk_size` — размер кусочка
- `chunk_overlap` — перекрытие, чтобы смысл не терялся на границе

## 4. Эмбеддинги — превращаем текст в числа

LLM не ищет напрямую по словам. Мы переводим текст в **вектор чисел**.

Если два текста похожи по смыслу — их вектора тоже «рядом».

В модуле это: `02_embeddings_retrievers.ipynb`.

Мы использовали:
- `CacheBackedEmbeddings` — чтобы не пересчитывать одно и то же,
- `Chroma` — как векторную базу.

## 5. Векторная база и хранение на диске

**Векторная база** хранит эмбеддинги и умеет быстро искать похожие.

Мы использовали **Chroma**, и научились хранить базу **на диске**, чтобы не пересоздавать:

```
persist_directory = "./db/chroma_rag"
```

Это было в `05_rag_pipeline.ipynb`.

## 6. Ретривер — «поисковик» для вашей базы

Ретривер — это часть, которая **находит нужные кусочки** текста.

Мы строили разные виды ретриверов:
- обычный (по смыслу),
- гибридный (с BM25),
- ParentDocumentRetriever (чтобы сохранять контекст).

Подробно — в блокнотах `02`–`04`.

## 7. Гибридный поиск (BM25 + Вектор)

Иногда ключевое слово важнее смысла.
Например, слово «ИНН» должно попасть точно.

Гибридный поиск соединяет:
- **BM25** — точные слова
- **Вектор** — смысл

Мы делали это в `03_hybrid_search.ipynb`.

## 8. ParentDocumentRetriever — большие ответы с контекстом

Если вернуть только маленький кусочек, ответ может быть «обрывочным».

ParentDocumentRetriever делает так:
1. Ищет по маленьким кускам,
2. Возвращает **родительский** документ побольше.

Это было в `04_parent_retriever.ipynb`.

## 9. Полный RAG‑пайплайн (как LLM получает знания)

Самая важная часть:

1. Пользователь пишет вопрос.
2. Ретривер ищет релевантные кусочки.
3. Эти кусочки добавляются в промпт.
4. LLM отвечает, читая **контекст**.

LLM **не ищет сама**, ей «подкладывают» нужные фрагменты.

Это основной блокнот: `05_rag_pipeline.ipynb`.

## 10. RAG по Markdown

Markdown — это текст с заголовками, списками и кодом.

Чтобы всё работало хорошо, мы:
- аккуратно чистили Markdown,
- сохраняли заголовки в метаданных.

Это делалось в `06_rag_from_md.ipynb`.

## 11. Реальные .md файлы (с кодом)

В реальной жизни в документах есть:
- кодовые блоки,
- HTML/CSS/XPath,
- технические куски.

Мы сделали специальный парсер, чтобы **не ломать код**.

Это блокнот: `07_rag_real_md.ipynb`.

## 12. Домашнее задание (Vector DB) — что мы делали

Мы повторили «большое» ДЗ:

- выбрали векторную БД,
- проиндексировали документы,
- сравнили скорость и точность,
- посмотрели параметры HNSW,
- построили график.

Это блокнот: `08_hw_vector_db.ipynb`.

## 13. Русский датасет (Lenta / ru_news)

Мы сделали русскую версию ДЗ:
- загрузка Lenta или ru_news без `datasets`,
- локальные эмбеддинги (чтобы не зависеть от API),
- фильтрация по метаданным,
- итоги и выводы.

Это блокнот: `09_hw_vector_db_lenta.ipynb`.

## 14. Мини‑чеклист (очень коротко)

1. Очистить данные.
2. Разбить на кусочки.
3. Сделать эмбеддинги.
4. Положить в векторную базу.
5. Настроить ретривер.
6. Добавить контекст в промпт.
7. Получить ответ.

Если вы это умеете — вы уже сделали RAG.

## 15. Что делать дальше

Если хотите углубиться:
- увеличить объём данных (10k+ документов),
- попробовать другой эмбеддер,
- сравнить разные параметры HNSW,
- добавить тесты качества.

Готов продолжить и сделать модуль 5 так же подробно.